In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
transections = "/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv"
articals = "/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv"
sample_submission = "/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv"
customers = "/kaggle/input/h-and-m-personalized-fashion-recommendations/customers.csv"

transactions_df = pd.read_csv(transections)
articles_df = pd.read_csv(articals)
customers_df = pd.read_csv(customers)

In [3]:
print(transactions_df.head())
print(articles_df.head())
print(customers_df.head())

        t_dat                                        customer_id  article_id  \
0  2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...   663713001   
1  2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...   541518023   
2  2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699...   505221004   
3  2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699...   685687003   
4  2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699...   685687004   

      price  sales_channel_id  
0  0.050831                 2  
1  0.030492                 2  
2  0.015237                 2  
3  0.016932                 2  
4  0.016932                 2  
   article_id  product_code          prod_name  product_type_no  \
0   108775015        108775          Strap top              253   
1   108775044        108775          Strap top              253   
2   108775051        108775      Strap top (1)              253   
3   110065001        110065  OP T-shirt (Idro)             

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


# Let's check some basic stats

In [4]:
# Convert `t_dat` to a datetime type for easy date manipulation
transactions_df['t_dat'] = pd.to_datetime(transactions_df['t_dat'])

# Get a high-level overview of the data
print("Transactions DataFrame Info:")
print(transactions_df.info())

# Count the number of unique customers and articles
num_unique_customers = transactions_df['customer_id'].nunique()
num_unique_articles = transactions_df['article_id'].nunique()

# Find the date range of the transactions
start_date = transactions_df['t_dat'].min()
end_date = transactions_df['t_dat'].max()

# Print the key statistics
print("\nBasic Statistics")
print(f"Number of unique customers: {num_unique_customers}")
print(f"Number of unique articles: {num_unique_articles}")
print(f"Date range of transactions: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

Transactions DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       object        
 2   article_id        int64         
 3   price             float64       
 4   sales_channel_id  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 1.2+ GB
None

Basic Statistics
Number of unique customers: 1362281
Number of unique articles: 104547
Date range of transactions: 2018-09-20 to 2020-09-22


# --- EDA on Customers Data ---
#### To understand the dataset better, the next logical step in our EDA is to visualize how the number of transactions changes over time. This can reveal seasonal patterns, trends, and key events that influence purchasing behavior.

In [5]:
print("--- Customers Dataframe EDA ---")
print("\nMissing values in customers data:")
print(customers_df.isnull().sum())
print("\nDistribution of club_member_status:")
print(customers_df['club_member_status'].value_counts())
print("\nDistribution of fashion_news_frequency:")
print(customers_df['fashion_news_frequency'].value_counts())

--- Customers Dataframe EDA ---

Missing values in customers data:
customer_id                    0
FN                        895050
Active                    907576
club_member_status          6062
fashion_news_frequency     16011
age                        15861
postal_code                    0
dtype: int64

Distribution of club_member_status:
club_member_status
ACTIVE        1272491
PRE-CREATE      92960
LEFT CLUB         467
Name: count, dtype: int64

Distribution of fashion_news_frequency:
fashion_news_frequency
NONE         877711
Regularly    477416
Monthly         842
Name: count, dtype: int64


Customers Data: We have some key categorical features like club_member_status and fashion_news_frequency that can be very powerful predictors. The high number of missing values for FN and Active suggests that for many customers, this information isn't available. We'll need a strategy to handle these missing values, possibly by treating them as their own category. The age column, while mostly complete, has some missing values we can impute later.

# --- EDA on Articles Data ---

In [6]:
print("\n--- Articles Dataframe EDA ---")
print("\nMissing values in articles data:")
print(articles_df.isnull().sum())
print("\nDistribution of product_group_name:")
print(articles_df['product_group_name'].value_counts())
print("\nDistribution of garment_group_name:")
print(articles_df['garment_group_name'].value_counts())


--- Articles Dataframe EDA ---

Missing values in articles data:
article_id                        0
product_code                      0
prod_name                         0
product_type_no                   0
product_type_name                 0
product_group_name                0
graphical_appearance_no           0
graphical_appearance_name         0
colour_group_code                 0
colour_group_name                 0
perceived_colour_value_id         0
perceived_colour_value_name       0
perceived_colour_master_id        0
perceived_colour_master_name      0
department_no                     0
department_name                   0
index_code                        0
index_name                        0
index_group_no                    0
index_group_name                  0
section_no                        0
section_name                      0
garment_group_no                  0
garment_group_name                0
detail_desc                     416
dtype: int64

Distribution of prod

Articles Data: This data is very clean! Nearly every column is complete, giving us a rich set of features to work with. The product_group_name and garment_group_name columns are particularly useful and show us the dominance of clothing categories like 'Garment Upper body' and 'Garment Lower body'.

### Let's merge 3 data frames with sample data

In [7]:
# Convert `t_dat` to datetime
transactions_df['t_dat'] = pd.to_datetime(transactions_df['t_dat'])

# Filter transactions for the last 3 months
end_date = transactions_df['t_dat'].max()
start_date_filtered = end_date - pd.DateOffset(months=3)
recent_transactions = transactions_df[transactions_df['t_dat'] >= start_date_filtered]

# Merge the dataframes
merged_df = pd.merge(recent_transactions, customers_df, on='customer_id', how='left')
merged_df = pd.merge(merged_df, articles_df, on='article_id', how='left')

# Display the information of the new, merged dataframe
print("Merged DataFrame Info (last 3 months):")
print(merged_df.info())

print("\nMerged DataFrame Head:")
merged_df.head()

Merged DataFrame Info (last 3 months):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4056792 entries, 0 to 4056791
Data columns (total 35 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   t_dat                         datetime64[ns]
 1   customer_id                   object        
 2   article_id                    int64         
 3   price                         float64       
 4   sales_channel_id              int64         
 5   FN                            float64       
 6   Active                        float64       
 7   club_member_status            object        
 8   fashion_news_frequency        object        
 9   age                           float64       
 10  postal_code                   object        
 11  product_code                  int64         
 12  prod_name                     object        
 13  product_type_no               int64         
 14  product_type_name             object       

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,t_dat,customer_id,article_id,price,sales_channel_id,FN,Active,club_member_status,fashion_news_frequency,age,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,2020-06-22,000346516dd355b40badca0c0f5f37a318ddae31f0e0f7...,816588001,0.022017,2,NaN,NaN,ACTIVE,NONE,25.0,...,Tops Fancy Jersey,D,Divided,2,Divided,53,Divided Collection,1005,Jersey Fancy,"Cropped, off-the-shoulder jersey top with a sm..."
1,2020-06-22,000346516dd355b40badca0c0f5f37a318ddae31f0e0f7...,669708001,0.016932,2,NaN,NaN,ACTIVE,NONE,25.0,...,Tops Woven,D,Divided,2,Divided,53,Divided Collection,1010,Blouses,Short off-the-shoulder top in an airy weave wi...
2,2020-06-22,0008a2dd68b9a347b6f6b6d567b48684d4a11e05a8b7cc...,811907001,0.012695,1,NaN,NaN,ACTIVE,NONE,19.0,...,Swimwear,B,Lingeries/Tights,1,Ladieswear,60,"Womens Swimwear, beachwear",1018,Swimwear,Fully lined bikini top with removable inserts ...
3,2020-06-22,000f4d22ea7b4fc94704b3e6b7fb225b4f9dde7560cd38...,856332001,0.025814,2,1.0,1.0,ACTIVE,Regularly,27.0,...,Premium Quality,C,Ladies Accessories,1,Ladieswear,64,Womens Shoes,1020,Shoes,"Leather slides with wide, crossover straps and..."
4,2020-06-22,000f4d22ea7b4fc94704b3e6b7fb225b4f9dde7560cd38...,589748001,0.011610,2,1.0,1.0,ACTIVE,Regularly,27.0,...,Belts,C,Ladies Accessories,1,Ladieswear,65,Womens Big accessories,1019,Accessories,Wide waist belt with a round metal buckle. Wid...


### Let's fill the null data

In [8]:
# Let's fill null with the median age of the customers
median_age = merged_df['age'].median()
merged_df['age'].fillna(median_age, inplace=True)

# Impute missing categorical values with a placeholder
merged_df['club_member_status'] = merged_df['club_member_status'].fillna('Unknown')
merged_df['fashion_news_frequency'] = merged_df['fashion_news_frequency'].fillna('Unknown')
merged_df['FN'] = merged_df['FN'].fillna(0)
merged_df['Active'] = merged_df['Active'].fillna(0)

print(merged_df[['club_member_status', 'fashion_news_frequency', 'FN', 'Active']].isna().sum())

# Create temporal features
merged_df['week'] = merged_df['t_dat'].dt.isocalendar().week.astype(int)
merged_df['day_of_week'] = merged_df['t_dat'].dt.dayofweek.astype(int)

merged_df[['t_dat', 'age', 'FN', 'Active', 'club_member_status', 'week', 'day_of_week']].head()


/tmp/ipykernel_13/2658978471.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['age'].fillna(median_age, inplace=True)


club_member_status        0
fashion_news_frequency    0
FN                        0
Active                    0
dtype: int64


,t_dat,age,FN,Active,club_member_status,week,day_of_week
0,2020-06-22,25.0,0.0,0.0,ACTIVE,26,0
1,2020-06-22,25.0,0.0,0.0,ACTIVE,26,0
2,2020-06-22,19.0,0.0,0.0,ACTIVE,26,0
3,2020-06-22,27.0,1.0,1.0,ACTIVE,26,0
4,2020-06-22,27.0,1.0,1.0,ACTIVE,26,0


In [9]:
# calculate recency, frequency, and monetary value for each customer
customer_features = merged_df.groupby('customer_id').agg(
    total_purchase = ('article_id', 'count'),
    last_purchase_date = ('t_dat', 'max')
)
customer_features["recency_days"] = (merged_df['t_dat'].max() - customer_features['last_purchase_date']).dt.days
customer_features.head()

,total_purchase,last_purchase_date,recency_days
customer_id,,,
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,1,2020-09-05,17
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,1,2020-07-08,76
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,1,2020-09-15,7
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a,4,2020-08-12,41
0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d8cd0c725276a467a2a,2,2020-09-14,8


In [10]:
# calculate popularity and average price for each article
artical_features = merged_df.groupby('article_id').agg(
    purchase_count = ('customer_id', 'count'),
    average_price = ('price', 'mean')
)
artical_features.head()

,purchase_count,average_price
article_id,,
108775015,3,0.003842
108775044,98,0.008117
110065001,5,0.006342
110065002,6,0.005743
110065011,3,0.007797


# Now it's time to prepare the data model
1. generate positive samples
2. generate negative samples
3. merge features
4. finalizing the datasets

In [11]:
# To make the process faster, let's work with a small sample of the merged data
sample_merged_df = merged_df.sample(n=100000, random_state=42).reset_index(drop=True)

In [12]:
# Create positive samples with a label of 1
positive_samples = sample_merged_df[['customer_id', 'article_id']].copy()
positive_samples['label'] = 1

In [13]:
# Negative Sampling: Get all unique article IDs
all_article_ids = sample_merged_df['article_id'].unique()

In [14]:
# generate negetive sample list
negative_samples_list = []
for customer in positive_samples['customer_id'].unique():
    customer_purchases = set(positive_samples[positive_samples['customer_id'] == customer]['article_id'])
    
    # Get articles not purchased by the customer
    non_purchased_articles = np.setdiff1d(all_article_ids, list(customer_purchases))
    
    # Randomly sample a few non-purchased items for each purchase
    num_neg_samples = min(len(non_purchased_articles), 4) # Take 4 negative samples for each positive
    if num_neg_samples > 0:
        neg_articles = np.random.choice(non_purchased_articles, num_neg_samples, replace=False)
        for neg_article in neg_articles:
            negative_samples_list.append([customer, neg_article, 0])

negative_samples = pd.DataFrame(negative_samples_list, columns=['customer_id', 'article_id', 'label'])

negative_samples.head()

,customer_id,article_id,label
0,dda4b507ba647aec8fe0c10767133f05cce8b847c0e1fb...,861373001,0
1,dda4b507ba647aec8fe0c10767133f05cce8b847c0e1fb...,801803001,0
2,dda4b507ba647aec8fe0c10767133f05cce8b847c0e1fb...,861451002,0
3,dda4b507ba647aec8fe0c10767133f05cce8b847c0e1fb...,904995001,0
4,db48cdc0f7cedaea840f50dc2e7b52dd84e0d67c48ac82...,698387022,0


In [15]:
# Now combine positive and negative samples
final_data = pd.concat([positive_samples, negative_samples], ignore_index=True)

# Merge our aggregated features
final_data = pd.merge(final_data, customer_features, on='customer_id', how='left')
final_data = pd.merge(final_data, artical_features, on='article_id', how='left')

# Show the final dataset structure
print("\nFinal Dataset Shape:")
print(final_data.shape)
print("\nDistribution of Labels:")
print(final_data['label'].value_counts())

print("Final Dataset for Modeling Head:")
final_data.head()









Final Dataset Shape:
(433492, 8)

Distribution of Labels:
label
0    333492
1    100000
Name: count, dtype: int64
Final Dataset for Modeling Head:


,customer_id,article_id,label,total_purchase,last_purchase_date,recency_days,purchase_count,average_price
0,dda4b507ba647aec8fe0c10767133f05cce8b847c0e1fb...,850917001,1,11,2020-09-20,2,3530,0.024704
1,db48cdc0f7cedaea840f50dc2e7b52dd84e0d67c48ac82...,641804002,1,57,2020-09-20,2,38,0.033022
2,087e522ecf2fe0bf21eb9df8f1d93d0159e7aacad094bc...,850241001,1,14,2020-08-29,24,394,0.016158
3,7efc9f365f8905b3b7112097822e6c1acaba16bc991972...,862482009,1,32,2020-09-14,8,652,0.031093
4,87fa826bdcd3470c14251cd6bc4377c6697f08b382d375...,688105016,1,5,2020-07-16,68,145,0.030448


# Now that we have our final dataset, with both positive and negative samples, our next step is to build and train a machine learning model. This is where we will use all the features we engineered to create a powerful ranking model.
Now that we have our final dataset, with both positive and negative samples, our next step is to build and train a machine learning model. This is where we will use all the features we engineered to create a powerful ranking model.

## Building the Ranking Model with LightGBM
#### A great choice for this type of structured, tabular data is a gradient boosting model. They are fast, efficient, and often perform very well in competitions. We'll use LightGBM, a popular and high-performance library.

The process will involve these key steps:

1. Splitting the Data: We will split our final dataset into a training set and a validation set.

2. Defining Features: We will identify the features that our model will use to make predictions.

3. Training the Model: We'll train a LightGBM classifier to predict the label (purchase or not) for each customer-article pair.

## We need to import sklearn and LightGBM model

In [16]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [17]:
# Define features and target
features = ['total_purchase', 'recency_days', 'purchase_count', 'average_price']
target = 'label'

# Drop any NaN values that might have been created during merging the datasets
final_data.dropna(subset=features, inplace=True)

X = final_data[features]
y = final_data[target]

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize and train the LightGBM model
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)

# Print a confirmation message
print("LightGBM model training complete!")
print("Model accuracy on validation set:", lgb_model.score(X_val, y_val))

[LightGBM] [Info] Number of positive: 80000, number of negative: 266793
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016536 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 806
[LightGBM] [Info] Number of data points in the train set: 346793, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.230685 -> initscore=-1.204446
[LightGBM] [Info] Start training from score -1.204446
LightGBM model training complete!
Model accuracy on validation set: 0.8013817921775338
